This is what I did. It runs as of now, but there seems to be some problems. It answers to the excercises, because:

ex 1: Patient keeps track of their own times

ex 2: Preparation has multiple parallel tracks.

ex 3: Hospital has sequential stuff that are dependent on each other

ex 4: stats are collected via Stats object

ex 5: triangular strea, read code comments

In [56]:
import simpy
import random
import numpy

In [176]:
class Hospital(object):
    '''
    Hospital with three rooms. Keeps track on availability of rooms
    '''
    def __init__(self, env, prep_capacity, op_capacity, rec_capacity, avg_prep_time, avg_op_time, avg_rec_time):
        self.env = env
        self.prep = Room(self, env, prep_capacity, avg_prep_time, 'preparation', 0)
        self.op = Room(self, env, op_capacity, avg_op_time, 'operation', 1)
        self.rec = Room(self, env, op_capacity, avg_op_time, 'recovery', 2)
        self.stats = Stats(self, env, 'stats', 3)
        self.rooms = [self.prep, self.op, self.rec, self.stats]
    def rooms_free(self, name):
        if name == 'preparation':
            return self.op.available_places() and self.rec.available_places()
        elif name == 'operation':
            return self.rec.available_places()
        return True
    def put(self, patient, room_id):
        # print('hospital')
        # print(self.rooms[room_id].put(patient))
        if room_id <=2:
            self.env.process(self.rooms[room_id].put(patient))
        else:
            self.rooms[room_id].put(patient)
    def set_previous_free(self, i):
        if i >= 0:
            self.rooms[i].people_in -= 1

In [177]:
class Room(object):
    '''
    Room keeps track of its own usage and throughput.
    '''
    def __init__(self, hospital, env, capacity, avg_time, name, index):
        # print('room init')
        self.hospital = hospital
        self.env = env
        self.capacity = capacity
        self.avg_time = avg_time
        self.patients_taken = 0
        self.patients_processed = 0
        self.active_time = 0
        # self.out = hospital.rooms[index + 1]
        # self.unit = simpy.Resource(env, capacity)
        self.id = index
        self.name = name
        self.people_in = 0
    def available_places(self):
        return self.people_in < self.capacity
    def put(self, patient):
        # print(f'Room named {self.name}')
        got_in = False
        patient.idle_times[self.name] = 0
        while not got_in:
            # print(self.available_places() and hospital.rooms_free(self.name))
            if self.available_places() and hospital.rooms_free(self.name):
                # print('got in')
                got_in = True
                self.people_in += 1
                patient.arrival_times[self.name] = self.env.now
                time_yield = random.expovariate(1/self.avg_time)
                hospital.set_previous_free(self.id - 1)
                self.patients_taken += 1
                yield self.env.timeout(time_yield)
                patient.departure_times[self.name] = self.env.now
                hospital.put(patient, self.id + 1)
                self.patients_processed += 1
            else:
                yield self.env.timeout(1)
                patient.idle_times[self.name] += 1

In [178]:
class Patient(object):
    '''
    Patient keeps track of arrival times to different places and idle times.
    '''
    def __init__(self, patient_id, arrival_time):
        self.patient_id = patient_id
        self.arrival_times = {}
        self.arrival_times['hospital'] = arrival_time
        self.departure_times = {}
        self.idle_times = {}
    

In [179]:
class PatientGenerator(object):
    '''
    Generates patients.
    '''
    def __init__(self, env,hospital, arr_interval):
        self.env = env
        self.arr_interval = arr_interval
        self.patient_id = 0
        self.hospital = hospital
        self.action = env.process(self.run())

    def run(self):
        while True:
            wait_time = random.expovariate(1/self.arr_interval) # Excercise 5: random.triangular(10,25,40)
            yield self.env.timeout(wait_time)
            print('one patient arrived')
            self.patient_id += 1 
            patient = Patient(self.patient_id, 
                              self.env.now)
            self.hospital.put(patient, 0)

In [180]:
class Stats(object):
    '''
    Stats keep a poll just outside the hospital. They want to know if you'd give them a smiley.
    '''
    def __init__(self, hospital, env, name, index):
        self.hospital = hospital
        self.env = env
        self.name = name
        self.id = index
        self.op_usage_minutes = 0
        self.rec_usage_minutes = 0
        self.prep_usage_minutes = 0
        self.op_usage_rate = 0
        self.rec_usage_rate = 0
        self.prep_usage_rate = 0
        self.average_idle = 0
        self.patients_out = 0
        self.total_idle = 0
        
    def put(self, patient):
        hospital.set_previous_free(self.id - 1)
        self.op_usage_minutes += patient.departure_times['operation'] - patient.arrival_times['operation']
        self.rec_usage_minutes += patient.departure_times['recovery'] - patient.arrival_times['recovery']
        self.prep_usage_minutes += patient.departure_times['preparation'] - patient.arrival_times['preparation']
        self.op_usage_rate = self.op_usage_minutes / (self.hospital.op.capacity * self.env.now)
        self.rec_usage_rate = self.rec_usage_minutes / (self.hospital.rec.capacity * self.env.now)
        self.prep_usage_rate = self.prep_usage_minutes / (self.hospital.prep.capacity * self.env.now)
        self.patients_out += 1
        self.total_idle += np.sum(list(patient.idle_times.values()))
        self.average_idle = self.total_idle / self.patients_out
    def print_stats(self):
        print(f'operation minutes {self.op_usage_minutes}')
        print(f'recovery minutes {self.rec_usage_minutes}')
        print(f'preparation minutes {self.prep_usage_minutes}')
        print(f'operation usage rate {self.op_usage_rate}')
        print(f'recovery usage rate {self.rec_usage_rate}')
        print(f'preparation usage rate {self.prep_usage_rate}')
        print(f'average idle {self.average_idle}')
        print(f'patients out {self.patients_out}')

In [181]:
prep_rooms = 3
op_room = 1
rec_rooms = 3
arr_interval = 25
avg_prep_time = 40
avg_op_time = 20
avg_rec_time = 40
max_time = 1000

env = simpy.Environment()
hospital = Hospital(env, prep_rooms, op_room, rec_rooms, avg_prep_time, avg_op_time, avg_rec_time)
patient_gen = PatientGenerator(env, hospital, arr_interval)
env.run(until=max_time)
print(hospital.stats.print_stats())


one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
one patient arrived
operation minutes 527.3385483370041
recovery minutes 332.43836405848424
preparation minutes 637.3104288702816
operation usage rate 0.5539240970089095
recovery usage rate 0.349198102818256
preparation usage rate 0.2231467210451476
average id

In [32]:
import numpy as np
np.sum(list({'a':1, 'b':2}.values()))

3

ex 6: Lehmer generator

In [99]:
import numpy as np
from scipy import stats

def lehmer(seed=10, max_len=10000, m=10e8+1, A=23, C=0):
    X = np.zeros(max_len)
    R = np.zeros(max_len)
    X[0] = seed
    R[0] = X[0] / m
    for n in range(1, max_len):
        X[n] = (A * X[n-1] + C) % m
        R[n] = X[n] / m
    return R

In [101]:
arr = lehmer()

In [167]:
arr3 = []
arr4 = []
for j in [10**(-k) for k in [0, 1,2,3,4]]:
    arr2 = []
    for i in arr:
        if i < j:
            arr2.append(i)
    arr3.append(np.average(arr2)* (1/j))
    arr4.append(np.average(arr2))
    

In [168]:
arr3 # Expect to have 0.5 in every one --> not random

[0.501048999967351,
 0.4895408288069416,
 0.465237501304674,
 0.43855929367908775,
 0.018433333314899998]

In [169]:
arr4

[0.501048999967351,
 0.04895408288069416,
 0.00465237501304674,
 0.00043855929367908776,
 1.84333333149e-06]